# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [41]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import time

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [42]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [43]:
from secrets import FINNHUB_API_KEY
print(FINNHUB_API_KEY)

clqjqa9r01qjl4hq3f1gclqjqa9r01qjl4hq3f20


## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [ ]:
# FINNHUB

In [4]:
symbol = 'AAPL'

pricing_api_url = f'https://finnhub.io/api/v1/quote?symbol={symbol}&token={FINNHUB_API_KEY}'
pricing_data = requests.get(pricing_api_url).json()
print(pricing_data)

market_cap_api_url = f'https://finnhub.io/api/v1/stock/profile2?symbol={symbol}&token={FINNHUB_API_KEY}'
market_cap_data = requests.get(market_cap_api_url).json()
print(market_cap_data)

{'c': 195.71, 'd': 1.44, 'dp': 0.7412, 'h': 195.99, 'l': 193.67, 'o': 194.2, 'pc': 194.27, 't': 1702069201}
{'country': 'US', 'currency': 'USD', 'estimateCurrency': 'USD', 'exchange': 'NASDAQ NMS - GLOBAL MARKET', 'finnhubIndustry': 'Technology', 'ipo': '1980-12-12', 'logo': 'https://static2.finnhub.io/file/publicdatany/finnhubimage/stock_logo/AAPL.svg', 'marketCapitalization': 3043829.09, 'name': 'Apple Inc', 'phone': '14089961010', 'shareOutstanding': 15552.75, 'ticker': 'AAPL', 'weburl': 'https://www.apple.com/'}


In [5]:
print("price: ")
print(pricing_data['c'])

print("market cap: ")
print(market_cap_data['marketCapitalization'])

price: 
195.71
market cap: 
3043829.09


In [ ]:
# ALPHA VANTAGE

In [4]:
symbol = 'AAPL'
pricing_api_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&apikey={ALPHA_VANTAGE_API_KEY}&symbol={symbol}&interval=60min'
pricing_data = requests.get(pricing_api_url).json()
print(pricing_data)

market_cap_api_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={ALPHA_VANTAGE_API_KEY}'
market_cap_data = requests.get(market_cap_api_url).json()
print(market_cap_data)

{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


In [4]:
print("price: ")
print(pricing_data['Time Series (60min)']['2023-12-08 16:00:00']['4. close'])

print("market cap: ")
print(market_cap_data['MarketCapitalization'])

{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [16]:
price = pricing_data['c']
print("price:", price)

market_cap = market_cap_data['marketCapitalization']/1000000
print("market cap:", market_cap, 'trillion')

price: 195.71
market cap: 3.04382909 trillion


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [45]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [17]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'

    ],
    index = my_columns
    ),
    ignore_index=True
)    

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,195.71,3.043829,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [46]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    pricing_api_url = f'https://finnhub.io/api/v1/quote?symbol={stock}&token={FINNHUB_API_KEY}'
    pricing_data = requests.get(pricing_api_url).json()
    price = pricing_data['c']
    if price == 0:
        price = 'N/A'
        market_cap = 'N/A'
    else:
        market_cap_api_url = f'https://finnhub.io/api/v1/stock/profile2?symbol={stock}&token={FINNHUB_API_KEY}'
        market_cap_data = requests.get(market_cap_api_url).json()
        market_cap = market_cap_data['marketCapitalization']/1000
    
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            price,
            market_cap,
            'N/A'
    
        ],
        index = my_columns
        ),
        ignore_index=True
    )
    time.sleep(1)

In [60]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.2,37.1581,155
1,AAL,13.76,8.99272,1439
2,AAP,56.27,3.34818,351
3,AAPL,195.71,3043.83,101
4,ABBV,149.28,263.559,132
...,...,...,...,...
500,YUM,124.34,34.8535,N/A
501,ZBH,117.3,24.5134,N/A
502,ZBRA,236.13,12.1275,N/A
503,ZION,38.93,5.76743,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [48]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [37]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f''



######## finnhub does not support batch API requests ########

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [53]:
portfolio_size = input('Enter the value of your portfolio:')

while True:
    try:
        portfolio_size = float(portfolio_size)
        break
    except:
        pass
        print("That's not a number! Please try again:")
        portfolio_size = input('Enter the value of your portfolio:')

Enter the value of your portfolio: 10000000


In [61]:
position_size = portfolio_size/len(final_dataframe.index)

print(type(position_size))
print(type(final_dataframe.loc[i, 'Stock Price']))

for i in range(0, len(final_dataframe.index)):
    if final_dataframe.loc[i, 'Stock Price'] == 'N/A':
        final_dataframe.loc[i, 'Number of Shares to Buy'] = 'N/A'
    else:
        final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/float(final_dataframe.loc[i, 'Stock Price']))

final_dataframe
# number_of_apple_shares = position_size/500
# print(math.floor(number_of_apple_shares))

<class 'float'>
<class 'str'>


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.2,37.1581,155
1,AAL,13.76,8.99272,1439
2,AAP,56.27,3.34818,351
3,AAPL,195.71,3043.83,101
4,ABBV,149.28,263.559,132
...,...,...,...,...
500,YUM,124.34,34.8535,159
501,ZBH,117.3,24.5134,168
502,ZBRA,236.13,12.1275,83
503,ZION,38.93,5.76743,508


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [90]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [91]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [92]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [93]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', dollar_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [ ]:
writer.save()